In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks, knockout_FBA_simple

from functools import partial
from src.FBA_scripts.met_task_functions import constrain_model, read_tasks


In [2]:
# Single model
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/pancreas.xml'
#model = cobra.io.read_sbml_model(model_file_path)

In [3]:
# Models for tasks
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpqhpj5l9q.lp
Reading time = 0.07 seconds
: 5597 rows, 15340 columns, 67174 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpdq6wlt_7.lp
Reading time = 0.07 seconds
: 5597 rows, 15340 columns, 67174 nonzeros


In [85]:
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
# Constrain model
task = task_list[-1]
for subset in [task[0], task[1]]:
    for rx in subset:
        if rx == 'ALLMETSIN':
            # Adds boundary metabolites for other reactions when ALLMETSIN is used
            for r in subset[1:]:
                for m2 in r.metabolites:
                    for r2 in m2.reactions:
                        if r2.boundary and r2.id != r.id:
                            # Could also just remove the reactions, or set them 0, 0
                            r2.add_metabolites({Metabolite(
                                                m2.id[:-4] + 'x[x]',
                                                formula=m2.formula,
                                                name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                compartment='x'): 1})
            continue
        model_list[3].add_reaction(rx)


In [16]:
# FBA + knockouts

res_list = []

In [ ]:
model = model_list[0]
res_list.append(round(knockout_FBA_simple(model, []), 3))
res_list.append(round(knockout_FBA_simple(model, ['ENSG00000134538']), 3))

In [114]:
model = model_list[3]
with model:
    # Bilirubin sink
    model.reactions.get_by_id('HMR_9273').lower_bound = -1000
    model.reactions.get_by_id('HMR_9273').upper_bound = -249

    # Bilirubin-Bisglucuronoside sink
    model.reactions.get_by_id('HMR_9271').lower_bound = 0
    model.reactions.get_by_id('HMR_9271').upper_bound = 0

    # Bilirubin-monoglucuronoside sink
    model.reactions.get_by_id('HMR_9272').lower_bound = 10
    model.reactions.get_by_id('HMR_9272').upper_bound = 1000

    # Heme synthesis
    model.reactions.get_by_id('HMR_4758').lower_bound = 0
    model.reactions.get_by_id('HMR_4758').upper_bound = 1

    # Oxygen
    model.reactions.get_by_id('HMR_9048').lower_bound = -10
    # Glucose
    model.reactions.get_by_id('HMR_9034').lower_bound = -6

    # Threonine
    model.reactions.get_by_id('HMR_9044').lower_bound = -1

    #model.objective = 'HMR_9271'
    #model.objective_direction = 'max'

    print(knockout_FBA_simple(model, []))
    print(knockout_FBA_simple(model, ['ENSG00000134538', 'ENSG00000184999', 'ENSG00000103222']))

2.1839945137303576
2.1839945137303576


In [7]:
res_list_lung = res_list

[82.17122442635676, 81.6253209998778]


In [18]:
print('Lung results: ' + str(res_list_lung))
print(res_list)


Lung results: [82.17122442635676, 81.6253209998778]
[111.339, 111.339]


In [117]:
# FBA + knockouts

res_list = []

In [2]:
tissue = 'liver'

model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=True)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp71gt85ib.lp
Reading time = 0.07 seconds
: 5788 rows, 15892 columns, 69376 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp3eaaadly.lp
Reading time = 0.07 seconds
: 5788 rows, 15892 columns, 69376 nonzeros


In [33]:
if False:
    genes = [gene.id for gene in model_list[0].genes]
else:
    genes = pd.read_table('C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/{0}_non_essential.tsv'.format(tissue), index_col=0)['gene_ids'].tolist()

# Create and run combinations for individual samples
t='ENSG00000154305;ENSG00000132677;ENSG00000140057;ENSG00000167280;ENSG00000105499;ENSG00000070159;ENSG00000088035;ENSG00000176463;ENSG00000077463;ENSG00000115884;ENSG00000167701;ENSG00000105929;ENSG00000198610;ENSG00000168350;ENSG00000188833;ENSG00000076555;ENSG00000213614;ENSG00000109861;ENSG00000171503;ENSG00000183421;ENSG00000157765;ENSG00000002726;ENSG00000165434;ENSG00000166819;ENSG00000198569;ENSG00000147804;ENSG00000138621;ENSG00000149177;ENSG00000064270;ENSG00000126432;ENSG00000104823;ENSG00000129465;ENSG00000065621;ENSG00000132958;ENSG00000071909;ENSG00000105426;ENSG00000124006;ENSG00000172890;ENSG00000165029;ENSG00000169359;ENSG00000114480;ENSG00000168038;ENSG00000125255;ENSG00000070915;ENSG00000206077;ENSG00000171124;ENSG00000134780;ENSG00000164543;ENSG00000115159;ENSG00000162408;ENSG00000111700;ENSG00000141736;ENSG00000277494;ENSG00000109854;ENSG00000160446;ENSG00000277893;ENSG00000155850;ENSG00000143776;ENSG00000008128;ENSG00000113492;ENSG00000173083;ENSG00000188906;ENSG00000095303;ENSG00000116791;ENSG00000136381;ENSG00000146085;ENSG00000148834;ENSG00000037280;ENSG00000132837;ENSG00000181555;ENSG00000125246;ENSG00000257335;ENSG00000023839;ENSG00000137124;ENSG00000143036;ENSG00000114923;ENSG00000132437;ENSG00000060237;ENSG00000162365;ENSG00000142798;ENSG00000145626;ENSG00000165996;ENSG00000072756;ENSG00000214013;ENSG00000102595;ENSG00000186204;ENSG00000170266;ENSG00000150457;ENSG00000134070;ENSG00000070669;ENSG00000131446;ENSG00000146904;ENSG00000184983;ENSG00000175164;ENSG00000114670;ENSG00000138604;ENSG00000100577;ENSG00000163959;ENSG00000109667;ENSG00000058729;ENSG00000139304;ENSG00000168907;ENSG00000132744;ENSG00000136877;ENSG00000135090;ENSG00000138166;ENSG00000171105;ENSG00000196730;ENSG00000174292;ENSG00000069020;ENSG00000115020;ENSG00000120156;ENSG00000065534;ENSG00000166035;ENSG00000204371;ENSG00000135372;ENSG00000116539;ENSG00000198074;ENSG00000167676;ENSG00000127415;ENSG00000112499;ENSG00000156966;ENSG00000141503;ENSG00000180953;ENSG00000184470;ENSG00000103485;ENSG00000134013;ENSG00000154229;ENSG00000175054;ENSG00000204385;ENSG00000143772;ENSG00000131748;ENSG00000117834;ENSG00000149124;ENSG00000198825;ENSG00000162482;ENSG00000134202;ENSG00000095015;ENSG00000162104;ENSG00000156222;ENSG00000159082;ENSG00000140474;ENSG00000172543;ENSG00000104763;ENSG00000038002;ENSG00000054983;ENSG00000112299;ENSG00000125779;ENSG00000145476;ENSG00000066230;ENSG00000090376;ENSG00000158104;ENSG00000177169;ENSG00000197586;ENSG00000174607;ENSG00000099810;ENSG00000110203;ENSG00000162882;ENSG00000170390;ENSG00000198130;ENSG00000164867;ENSG00000196139;ENSG00000133256;ENSG00000204084;ENSG00000165140;ENSG00000129219;ENSG00000135318;ENSG00000181192;ENSG00000168827;ENSG00000181409;ENSG00000115286;ENSG00000172071;ENSG00000197093;ENSG00000119227;ENSG00000274391;ENSG00000164879;ENSG00000160867;ENSG00000131459;ENSG00000159433;ENSG00000116711;ENSG00000087253;ENSG00000086015;ENSG00000173540;ENSG00000119927;ENSG00000177239;ENSG00000196616;ENSG00000074803;ENSG00000197355;ENSG00000186716;ENSG00000106123;ENSG00000275342;ENSG00000085511;ENSG00000087586;ENSG00000130653;ENSG00000143882;ENSG00000156510;ENSG00000147408;ENSG00000145217;ENSG00000149476;ENSG00000187486;ENSG00000085998;ENSG00000078070;ENSG00000105355;ENSG00000058866;ENSG00000112303;ENSG00000123989'
input = t.split(';')
print('Num genes: ' + str(len(input)))
input = list(set(input).intersection(genes))
print(input)

combs = []
for i in range(1, 3): #len(input) + 1):
    for c in itertools.combinations(input, i):
        combs.append(list(c))

combs.append(input)
test_df = pd.DataFrame({'gene_ids': combs})

print('Num combs: ' + str(len(combs)))

task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks_min_ess_aa.tsv', model_list)
# Adds empty task result list column
test_df['tasks_results'] = np.empty((len(test_df), 0)).tolist()
test_df.reset_index(inplace=True)
test_df.rename(columns={'index': 'sample_ids'}, inplace=True)
test_df['sample_ids'] = test_df['sample_ids'].apply(lambda x: x + 1)

# Actual FBA
print(test_df.shape[0])
#print(test_df)

Num genes: 206
['ENSG00000171124', 'ENSG00000104823', 'ENSG00000114480', 'ENSG00000078070', 'ENSG00000167701', 'ENSG00000109861', 'ENSG00000172890', 'ENSG00000155850', 'ENSG00000115020', 'ENSG00000090376', 'ENSG00000115884', 'ENSG00000038002', 'ENSG00000132958', 'ENSG00000137124', 'ENSG00000105426', 'ENSG00000124006', 'ENSG00000134780', 'ENSG00000177239', 'ENSG00000181555', 'ENSG00000181409', 'ENSG00000154305', 'ENSG00000134070', 'ENSG00000109854', 'ENSG00000188833', 'ENSG00000204385', 'ENSG00000077463', 'ENSG00000129219', 'ENSG00000168350', 'ENSG00000115159', 'ENSG00000154229', 'ENSG00000112303', 'ENSG00000054983', 'ENSG00000213614', 'ENSG00000196616', 'ENSG00000171503', 'ENSG00000168827', 'ENSG00000169359', 'ENSG00000274391', 'ENSG00000114670', 'ENSG00000162882', 'ENSG00000116539', 'ENSG00000114923', 'ENSG00000142798', 'ENSG00000275342', 'ENSG00000198825', 'ENSG00000143772', 'ENSG00000131446', 'ENSG00000135372', 'ENSG00000058729', 'ENSG00000132437', 'ENSG00000156510', 'ENSG0000017716

In [34]:
start_time = time.time()

results = parallelize_dataframe(test_df, partial(knockout_FBA_w_tasks, task_list, model_list), n_cores=12)

end_time = time.time()
print('Total time: %.6f seconds' % (end_time - start_time))

results.loc[-1] = ['REF', [], round(model_list[0].slim_optimize(error_value='nan'), 3), [1]]

# Put REF on top
results.index = results.index + 1  # shifting index
results.sort_index(inplace=True)

results['gene_ids'] = results['gene_ids'].apply(';'.join)
results['tasks_results'] = results['tasks_results'].apply(lambda x: x if not all(x) else ['ALL PASS'])

results.reset_index(inplace=True, drop=True)
results[['sample_ids', 'gene_ids', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_results/filtered/start_stop_het/subcombs/HG03963_double_{0}.tsv'.format(tissue), sep='\t')

Total time: 454.658170 seconds


In [23]:
gene_list = 'ENSG00000157184'.split(';')

print('           Base: ' + str(knockout_FBA_simple(model_list[0], [])))
for gene in gene_list:
    print(gene + ': ' + str(knockout_FBA_simple(model_list[0], [gene])))


           Base: 75.00989279272778
ENSG00000157184: <Solution 71.936 at 0x1fa8bb17f40>


In [29]:
tissue = 'skin'

model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
model_list = constrain_model(model_file_path, ALLMETSIN_OUT=False)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpip1nq42k.lp
Reading time = 0.07 seconds
: 5626 rows, 15548 columns, 68020 nonzeros


In [30]:
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_minus_ess.tsv', model_list)
# ['HIBCH', ['ENSG00000198130'], 0, []],['BCAT2', ['ENSG00000105552'], 0, []]

#res = pd.DataFrame([['REF', [], 0, []]], columns=['sample_ids', 'gene_ids', 'solution', 'tasks_results'])

#print(res)

In [36]:
tissues = ['pancreas', 'spleen', 'adipose_tissue', 'adrenal_gland', 'pituitary', 'thyroid', 'blood', 'brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve', 'lung', 'skin']

for tissue in tissues:
    file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/tissue_full_tasks/backup/full_tasks_minus_ess_{0}.tsv'.format(tissue)
    tasks_df_t = pd.read_table(file_path_t, index_col=0)

    print(tissue)
    print(tasks_df_t.shape[0])
    tasks_df_t = tasks_df_t[tasks_df_t['REF'] == 1].drop(['REF', 'LBequ_2', 'LBequ_3', 'UBequ_2', 'UBequ_3'], axis=1)

    #print(tasks_df_t.head())
    print(tasks_df_t.shape[0])

    print('______________________')
    tasks_df_t.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/tissue_full_tasks/full_tasks_minus_ess_{0}.tsv'.format(tissue), sep='\t')

pancreas
221
182
______________________
spleen
222
178
______________________
adipose_tissue
222
182
______________________
adrenal_gland
222
182
______________________
pituitary
222
183
______________________
thyroid
222
181
______________________
blood
220
176
______________________
brain
221
181
______________________
heart
220
182
______________________
kidney
222
183
______________________
liver
222
189
______________________
muscle
221
182
______________________
nerve
222
182
______________________
lung
222
179
______________________
skin
222
182
______________________


In [65]:
# make clean outputs.
tissue_list = ['brain']#['pancreas', 'spleen', 'adipose_tissue', 'adrenal_gland', 'pituitary', 'thyroid', 'blood',
               #'heart', 'kidney', 'liver', 'muscle', 'nerve', 'lung']
tissue_list.sort()

for tissue in tissue_list:

    input_file = 'C:/Users/Sigve/Genome_Data/results/ind_results/filtered/start_stop_het/full_tasks/ind_results_f_{0}.tsv'.format(tissue)
    output_file = 'C:/Users/Sigve/Genome_Data/results/ind_results/filtered/start_stop_het/full_tasks/ind_results_f_clean_{0}.tsv'.format(tissue)

    res = pd.read_table(input_file, index_col=0)
    res = res[res['tasks_results'] != "['ALL PASS']"]
    res['tasks_results'] = res['tasks_results'].apply(lambda x: [int(i) for i in x[1:-1].split(', ')])

    dict = {}
    for i, data in res.iterrows():
        dict[data['sample_ids']] = data['tasks_results']

    tasks_df_r = pd.DataFrame(dict)
    #print(tasks_df_r)

    file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/tissue_full_tasks/full_tasks_minus_ess_{0}.tsv'.format(tissue)
    tasks_df_t = pd.read_table(file_path_t, index_col=0).reset_index()
    tasks_df_t['task_id'] = tasks_df_t['index']#
    tasks_df_t['_'*89 + 'description'] = tasks_df_t['description'].apply(lambda x:  '_'*(100-len(x)) + x)
    tasks_df_t = tasks_df_t[['task_id', '_'*89 + 'description']]

    tasks_df_t = tasks_df_t.merge(tasks_df_r, right_index=True, left_index=True)

    tasks_df_t.to_csv(path_or_buf=output_file, sep='\t')


   task_id  \
0        1   
1        5   
2        6   
3        7   
4        8   

  _________________________________________________________________________________________description  
0  ______________________________________________...                                                    
1  ______________________________________________...                                                    
2  ______________________________________________...                                                    
3  ______________________________________________...                                                    
4  ______________________________________________...                                                    
     task_id  \
0          1   
1          5   
2          6   
3          7   
4          8   
..       ...   
176      222   
177      228   
178      229   
179      230   
180      234   

    ________________________________________________________________________________________

In [ ]:
res.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/FBA_test_pancreas.tsv', sep='\t')

In [63]:
file_path_t = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_minus_ess.tsv'
tasks_df_t = pd.read_table(file_path_t, index_col=0)

task_res = res.iat[0, 3]
print(task_res)
i = 0
for r in task_res:
    print('Task result: ' + str(r) +' :' + tasks_df_t.iat[i, 0])
    i+=1

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
Task result: 1 :Aerobic rephosphorylation of ATP from a fatty acid
Task result: 1 :Anaerobic rephosphorylation of ATP
Task result: 1 :Anaerobic rephosphorylation of GTP
Task result: 1 :Anaerobic rephosphorylation of CTP
Task result: 1 :Anaerobic rephosphorylation of UTP
Task result: 1 :ATP de novo synthesis (minimal substrates, physio

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
model = model_list[0]

with model:
    for rx in model.reactions:
        if rx.bounds[0] < 0:
            rx.lower_bound = -1

        if rx.bounds[1] > 0:
            rx.upper_bound = 1

    # Oxygen
    model.reactions.get_by_id('HMR_9048').lower_bound = -10
    # Glucose
    model.reactions.get_by_id('HMR_9034').lower_bound = -6
    # CO2
    model.reactions.get_by_id('HMR_9058').upper_bound = 10
    # Water
    model.reactions.get_by_id('HMR_9047').lower_bound = -10
    model.reactions.get_by_id('HMR_9047').upper_bound = 10

    model.reactions.get_by_id('GLYVALHISt').lower_bound = 0
    model.reactions.get_by_id('GLYVALHISt').upper_bound = 1


    base = model.optimize()
    print(flux_variability_analysis(model, model.reactions.get_by_id('HMR_9046'), loopless=True))
    #print(model.summary())
    print(base)
    print(model.metabolites.get_by_id('m03135c[c]').summary())
    print('------------------------------------------------------------')
    t_model = model
    with t_model:
        #kn = knockout_FBA(t_model, [gene])
        t_model.genes.get_by_id(gene).knock_out()

        print(flux_variability_analysis(t_model, t_model.reactions.get_by_id('HMR_9046'), loopless=True))
        kn = t_model.optimize()

        print(kn)
        print(t_model.metabolites.get_by_id('m03135c[c]').summary())
        #print(t_model.summary())


In [3]:
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.mat'
#model = cobra.io.read_sbml_model(model_file_path)
model = cobra.io.load_matlab_model(model_file_path)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [12]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.loopless import add_loopless, loopless_solution

#model = model_list[0]

gene = 'ENSG00000198130'

with model:


    rx_list = [model.reactions.get_by_id('MAR09046'), model.reactions.get_by_id('MAR03744')]

    #model.objective = model.reactions.get_by_id('MAR03744')

    #model.reactions.MAR03747.lower_bound = 10
    #model.reactions.MAR03747.upper_bound = 10

    # Force Valine uptake
    model.reactions.MAR09046.lower_bound = 10

    # Limit intermediate exports
    model.reactions.MAR09011.upper_bound = 10
    model.reactions.MAR11481.upper_bound = 10
    model.reactions.MAR11818.upper_bound = 10
    model.reactions.MAR10185.upper_bound = 10

    model.reactions.MAR09058.lower_bound = 1000

    # Oxygen
    model.reactions.MAR09048.lower_bound = -10
    # Glucose
    model.reactions.MAR09034.lower_bound = -6

    base = model.optimize()
    print(flux_variability_analysis(model, rx_list, loopless=True))

    print(base)
    print(model.metabolites.get_by_id('MAM03135c').summary())
    print(model.summary())
    print('------------------------------------------------------------')
    t_model = model
    with t_model:
        #kn = knockout_FBA(t_model, [gene])
        # t_model.reactions.get_by_id('HMR_3747')
        t_model.genes.get_by_id(gene).knock_out()
        t_rx_list = [t_model.reactions.get_by_id('MAR09046'), t_model.reactions.get_by_id('MAR03744')]
        print(flux_variability_analysis(t_model, t_rx_list, loopless=True))
        kn = t_model.optimize()

        print(kn)
        print(t_model.metabolites.get_by_id('MAM03135c').summary())
        print(t_model.summary())

# HIBCH deficient model is forced to export isobutyrylglycine or earlier metabolite.
# The non-deficient can break it valine down.


          minimum  maximum
MAR09046     10.0   1000.0
MAR03744      0.0   1000.0
<Solution 187.354 at 0x1f2023ecee0>
MAM03135c
Formula: C5H11NO2

Producing Reactions
-------------------
Percent  Flux Reaction                                              Definition
 25.53% 19.78 MAR11220 2.0 MAM02040c + MAM04070c <=> MAM03089c + 2.0 MAM03135c
 74.47% 57.72 MAR11221                 MAM02040c + MAM04071c <=> 2.0 MAM03135c

Consuming Reactions
-------------------
Percent    Flux Reaction                                                                      Definition
 86.07%   -66.7 MAR05150         MAM01371c + MAM03082c + MAM03135c --> MAM01334c + MAM02423c + MAM02759c
 12.90%     -10 MAR05574                                 MAM02184c + MAM03135s <=> MAM02184s + MAM03135c
  1.03% -0.7972 MAR10064 0.0043 MAM00180c + 0.0014 MAM00569c + 0.0022 MAM00913c + 0.0025 MAM00916c + ...
Objective
1.0 MAR13082 = 187.35362997658078

Uptake
------
Metabolite Reaction      Flux  C-Number C-Flux
 MAM01291s

In [57]:
#task_list_fix
file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_old.tsv'

tasks_df = pd.read_table(file_path, index_col=0)
print(tasks_df.shape[0])
tasks_df.drop(29, inplace=True)
tasks_df = tasks_df[tasks_df['shouldFail'].map(lambda x: x != 1)]
tasks_df.drop(['shouldFail', 'printFluxes', 'comments', 'changed', 'LBrxn', 'UBrxn'], axis=1, inplace=True) #  'LBequ_2', 'LBequ_3',  'UBequ_2', 'UBequ_3'
tasks_df.rename({'LBequ_1': 'LBequ', 'UBequ_1': 'UBequ'}, inplace=True, axis=1)
tasks_df.reset_index(inplace=True, drop=True)
print(tasks_df.shape[0])
tasks_df.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks.tsv', sep='\t')

257
240


In [9]:
print(tasks_df.iloc[28, :])

description                     Release of glucose from Glycogen
inputs                                 'glycogenin G11[c]H2O[s]'
LBin                                                         1,0
UBin                                                      1,1000
outputs                                'glucose[s]glycogenin[c]'
LBout                                                      11, 1
UBout                                                      11, 1
equations      glucose-1-phosphate[c] + H2O[c] => glucose[c] ...
LBequ                                                        0.0
LBequ_2                                                      NaN
LBequ_3                                                      NaN
UBequ                                                     1000.0
UBequ_2                                                      NaN
UBequ_3                                                      NaN
Name: 28, dtype: object


In [58]:
file_path_1 = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv'
tasks_df_1 = pd.read_table(file_path_1, index_col=0)

file_path_2 = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks.tsv'
tasks_df_2 = pd.read_table(file_path_2, index_col=0)

description_list = tasks_df_1['description'].tolist()

print(tasks_df_2.shape[0])
tasks_df_2 = tasks_df_2[tasks_df_2['description'].map(lambda x: not x in description_list)]
print(tasks_df_2.shape[0])

tasks_df_2.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/full_tasks_minus_ess.tsv', sep='\t')


240
222


In [ ]:
#


In [ ]:
file_path_t ='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/valine_degradation_task.tsv'
tasks_df_t = read_tasks(file_path_t, model_list)

tissues = ['pancreas', 'spleen', 'adipose_tissue', 'adrenal_gland', 'pituitary', 'thyroid', 'blood', 'brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve', 'lung', 'skin']
# , ['HIBCH', ['ENSG00000198130'], 0, []],['BCAT2', ['ENSG00000105552'], 0, []]

res = pd.DataFrame([['skin', [], 0, []]], columns=['sample_ids', 'gene_ids', 'solution', 'tasks_results'])
res = knockout_FBA_w_tasks(tasks_df_t, model_list, res)

for tissue in tissues:
    model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'.format(tissue)
    model_list = constrain_model(model_file_path, ALLMETSIN_OUT=False)

    tasks_df_t = read_tasks(file_path_t, model_list)

    res_t = pd.DataFrame([[tissue, [], 0, []]], columns=['sample_ids', 'gene_ids', 'solution', 'tasks_results'])
    res = pd.concat([res, knockout_FBA_w_tasks(tasks_df_t, model_list, res_t)])

print(res)

Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmproz5_84e.lp
Reading time = 0.13 seconds
: 5597 rows, 15340 columns, 67174 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp_zm3yk0p.lp
Reading time = 0.08 seconds
: 5626 rows, 15476 columns, 67870 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpn4v5v8hj.lp
Reading time = 0.07 seconds
: 5621 rows, 15492 columns, 67494 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpx9aa1ip5.lp
Reading time = 0.12 seconds
: 5668 rows, 15622 columns, 68500 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpq27fonhw.lp
Reading time = 0.12 seconds
: 5592 rows, 15544 columns, 68256 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpz02hep3e.lp
Reading time = 0.13 seconds
: 5685 rows, 15774 columns, 69184 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpn1ia79ep.lp
Reading time =

In [ ]:
res.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/valine_degradation_task_test.tsv', sep='\t')

